In [1]:
from pycaret.time_series import *

import pandas as pd
from pandas import Series, DataFrame
import numpy as np
import category_encoders as ce

# Visualization
import matplotlib.pylab as plt
from matplotlib import font_manager, rc
import matplotlib
import seaborn as sns
import plotly.express as px
%matplotlib inline
matplotlib.rcParams['font.family'] = 'Malgun Gothic'
# Preprocessing & Feature Engineering
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, LabelEncoder
from sklearn.impute import SimpleImputer 
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.feature_selection import SelectPercentile

# Hyperparameter Optimization
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV

# Modeling
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier, XGBRegressor, XGBRFRegressor
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import StackingClassifier
from sklearn.base import ClassifierMixin

# PyTorch
# import torch
# from torch.utils.data import Dataset, DataLoader, TensorDataset
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim
# from torch.autograd import Variable
# from torch.nn import Parameter
# from torch import Tensor
# from torch.utils.data import DataLoader

# Evaluation
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.metrics import log_loss

# Utility
import os
import time
import datetime
import random
import warnings; warnings.filterwarnings("ignore")
from IPython.display import Image
import pickle
from tqdm import tqdm
import platform
from itertools import combinations
from scipy.stats.mstats import gmean

# from bayes_opt import BayesianOptimization
# from num2words import num2words
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.outliers_influence import OLSInfluence

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
inter = pd.read_csv('../data/international_trade.csv')
submit = pd.read_csv('../data/sample_submission.csv')

# train.drop(columns = 'ID', inplace = True)
# test.drop(columns = 'ID', inplace = True)

In [3]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [4]:
train = train.rename(columns={'supply(kg)':'supply','price(원/kg)':'price'})
train.columns

Index(['ID', 'timestamp', 'item', 'corporation', 'location', 'supply',
       'price'],
      dtype='object')

In [5]:
# prepare the data
'''
0. item, location. corporation을 LabelEncoding한다.
1. item, location, corporation 을 time-series 컬럼으로 결합한다.
2. timestamp로부터 month, year, day, dayofweek 를 추출
'''

data = train.copy()
test_ = test.copy()
data['timestamp'] = pd.to_datetime(data['timestamp'])
test_['timestamp'] = pd.to_datetime(test_['timestamp'])

# 0.
encoders = {}
for col in ['item','location','corporation']:
    encoder = LabelEncoder()
    data[col] = encoder.fit_transform(data[col])
    test_[col] = encoder.transform(test_[col])
    encoders[col] = encoder

# 1.
data['item'] = ['item_' + str(i) for i in data['item']]
data['location'] = ['location_' + str(i) for i in data['location']]
data['corporation'] = ['corporation_' + str(i) for i in data['corporation']]
data['time_series'] = data[['item'
                            , 'corporation','location'
                           ]].apply(lambda x: '_'.join(x), axis=1)
data.drop(['item', 'corporation','location'], axis=1, inplace=True)
test_['item'] = ['item_' + str(i) for i in test_['item']]
test_['location'] = ['location_' + str(i) for i in test_['location']]
test_['corporation'] = ['corporation_' + str(i) for i in test_['corporation']]
test_['time_series'] = test_[['item'
                            , 'corporation','location'
                           ]].apply(lambda x: '_'.join(x), axis=1)
test_.drop(['item', 'corporation','location'], axis=1, inplace=True)


# 2.
data['year'] = data['timestamp'].dt.year
data['month'] = data['timestamp'].dt.month
data['day'] = data['timestamp'].dt.day
data['dow'] = data['timestamp'].dt.dayofweek
data['doy'] = data['timestamp'].dt.dayofyear
test_['year'] = test_['timestamp'].dt.year
test_['month'] = test_['timestamp'].dt.month
test_['day'] = test_['timestamp'].dt.day
test_['dow'] = test_['timestamp'].dt.dayofweek
test_['doy'] = test_['timestamp'].dt.dayofyear

data.head()


,ID,timestamp,supply,price,time_series,year,month,day,dow,doy
0,TG_A_J_20190101,2019-01-01,0.0,0.0,item_4_corporation_0_location_0,2019,1,1,1,1
1,TG_A_J_20190102,2019-01-02,0.0,0.0,item_4_corporation_0_location_0,2019,1,2,2,2
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,item_4_corporation_0_location_0,2019,1,3,3,3
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,item_4_corporation_0_location_0,2019,1,4,4,4
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,item_4_corporation_0_location_0,2019,1,5,5,5


In [6]:
# check the unique time_series
data['time_series'].nunique()

39

In [42]:
def combine_plotly_figs_to_html(plotly_figs, html_fname, include_plotlyjs='cdn', 
                                separator=None, auto_open=False):
    with open(html_fname, 'w',encoding='utf-8') as f:
        f.write(plotly_figs[0].to_html(full_html=True))#,include_plotlyjs=include_plotlyjs))
        for fig in plotly_figs[1:]:
            if separator:
                f.write(separator)
            f.write(fig.to_html(full_html=True))#, include_plotlyjs=False))

    if auto_open:
        import pathlib, webbrowser
        uri = pathlib.Path(html_fname).absolute().as_uri()
        webbrowser.open(uri)

In [43]:
# plot multiple time series with moving avgs in a loop**

import plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

figs = []
### 📢 30일 "주기"로 moving_average를 설정하여 item별로 그래프화한 것
for iters,i in enumerate(data['time_series'].unique()):
    subset = data[data['time_series'] == i]
    subset['moving_average'] = subset['price'].rolling(30).mean()
    ax = px.line(subset, x="timestamp", y=["price","moving_average"], title = i, template = 'plotly_dark')
    
    figs.append(ax)

plot_html_fname='../graphs_images/price_combined_plots.html'
combine_plotly_figs_to_html(figs, plot_html_fname)

In [ ]:
data.head(2)

In [7]:
all_time_series = data['time_series'].unique()
all_time_series

array(['item_4_corporation_0_location_0',
       'item_4_corporation_0_location_1',
       'item_4_corporation_1_location_0',
       'item_4_corporation_1_location_1',
       'item_4_corporation_2_location_0',
       'item_4_corporation_2_location_1',
       'item_4_corporation_3_location_0',
       'item_4_corporation_3_location_1',
       'item_4_corporation_4_location_0',
       'item_4_corporation_4_location_1',
       'item_2_corporation_0_location_0',
       'item_2_corporation_1_location_0',
       'item_2_corporation_2_location_0',
       'item_2_corporation_3_location_0',
       'item_2_corporation_3_location_1',
       'item_2_corporation_4_location_0',
       'item_2_corporation_4_location_1',
       'item_1_corporation_0_location_0',
       'item_1_corporation_0_location_1',
       'item_1_corporation_3_location_0',
       'item_1_corporation_4_location_0',
       'item_3_corporation_0_location_0',
       'item_3_corporation_0_location_1',
       'item_3_corporation_2_locat

In [11]:
data.columns, test_.columns

(Index(['ID', 'timestamp', 'supply', 'price', 'time_series', 'year', 'month',
        'day', 'dow', 'doy'],
       dtype='object'),
 Index(['ID', 'timestamp', 'item', 'corporation', 'location'], dtype='object'))

In [45]:
test_

,ID,timestamp,time_series,year,month,day,dow,doy
0,TG_A_J_20230304,2023-03-04,item_4_corporation_0_location_0,2023,3,4,5,63
1,TG_A_J_20230305,2023-03-05,item_4_corporation_0_location_0,2023,3,5,6,64
2,TG_A_J_20230306,2023-03-06,item_4_corporation_0_location_0,2023,3,6,0,65
3,TG_A_J_20230307,2023-03-07,item_4_corporation_0_location_0,2023,3,7,1,66
4,TG_A_J_20230308,2023-03-08,item_4_corporation_0_location_0,2023,3,8,2,67
...,...,...,...,...,...,...,...,...
1087,RD_F_J_20230327,2023-03-27,item_3_corporation_5_location_0,2023,3,27,0,86
1088,RD_F_J_20230328,2023-03-28,item_3_corporation_5_location_0,2023,3,28,1,87
1089,RD_F_J_20230329,2023-03-29,item_3_corporation_5_location_0,2023,3,29,2,88
1090,RD_F_J_20230330,2023-03-30,item_3_corporation_5_location_0,2023,3,30,3,89


In [ ]:
# columns_use = ['year','month','day','dow','doy']

In [17]:
data[data['time_series']=='item_3_corporation_5_location_0']

,ID,timestamp,supply,price,time_series,year,month,day,dow,doy
57874,RD_F_J_20190101,2019-01-01,0.0,0.0,item_3_corporation_5_location_0,2019,1,1,1,1
57875,RD_F_J_20190102,2019-01-02,0.0,0.0,item_3_corporation_5_location_0,2019,1,2,2,2
57876,RD_F_J_20190103,2019-01-03,411480.0,395.0,item_3_corporation_5_location_0,2019,1,3,3,3
57877,RD_F_J_20190104,2019-01-04,370960.0,437.0,item_3_corporation_5_location_0,2019,1,4,4,4
57878,RD_F_J_20190105,2019-01-05,387140.0,425.0,item_3_corporation_5_location_0,2019,1,5,5,5
...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,item_3_corporation_5_location_0,2023,2,27,0,58
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,item_3_corporation_5_location_0,2023,2,28,1,59
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,item_3_corporation_5_location_0,2023,3,1,2,60
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,item_3_corporation_5_location_0,2023,3,2,3,61


In [21]:
# item 별로 setup, train, predict
all_results=[]
final_models = {}

for i in tqdm(all_time_series):
    df_subset = data[data['time_series']==i]
    
    s = setup(data = df_subset,
              target='price',
              index='timestamp',
              ignore_features=['ID', 'supply',
                                'time_series','year','month','day'],
              # 결측값 없으므로 imputation은 생략
#               transform_exogenous='log',
              scale_target='minmax',
              scale_exogenous='minmax',
              fold_strategy='rolling',
              fold=3,
              fh=28,
              sp_detection='auto',
#               max_sp_to_consider='31',
              seasonality_type='auto',
              enforce_exogenous=True,
              n_jobs=-1,
              use_gpu=True,
              html=True,
              session_id=123,
              verbose=True
             )
    
    best_models = compare_models(sort='RMSE',verbose=False)
#     blended = blend_models(best_top3_models)
    
    p = pull().iloc[0:1]
    p['time_series'] = str(i)
    all_results.append(p)
    
#     f = finalize_model(blended)
    f = finalize_model(best_models)
    
    final_models[i] = f
    save_model(f,f"../models/ts_{i}_model")
    
    

  0%|                                                                                           | 0/39 [00:00<?, ?it/s]

,Description,Value
0,session_id,123
1,Target,price
2,Approach,Univariate
3,Exogenous Variables,Present
4,Original data shape,"(1523, 3)"
5,Transformed data shape,"(1523, 3)"
6,Transformed train set shape,"(1495, 3)"
7,Transformed test set shape,"(28, 3)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves

[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: gfx1035, Vendor: Advanced Micro Devices, Inc.
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves

...

[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves 

[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves 

[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves 

  0%|                                                                                           | 0/39 [11:50<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
concat_results = pd.concat(all_results,axis=0)
concat_results

In [18]:
all_scores = []
test_features = ['timestamp','dow','doy'] # ['year','month','day','dow','doy']

for i in tqdm(data['time_series'].unique()):
    test_subset = test_[test_['time_series']==i]
    test_subset = test_subset[test_features].set_index('timestamp')
    l = load_model(f'../models/ts_{i}_model',verbose=False)
    predict = predict_model(l,fh=28,X=test_subset)
    predict['time_series']=i
    all_scores.append(predict)

concat_df = pd.concat(all_scores,axis=0)
concat_df


  5%|████▎                                                                              | 2/39 [00:03<01:04,  1.74s/it]


TypeError: '<' not supported between instances of 'Period' and 'numpy.int64'

In [ ]:
predict = concat_df['prediction_label']
predict.describe()

In [ ]:
submit['answer'] = predict
submit

In [ ]:
submit.set_index('ID').to_csv('../data/pycaret_tsf_231116_0156.csv')